In [1]:
import tensorflow as tf
import numpy as np
import os
import scipy.io as spio
from matplotlib import pyplot as plt
from imageio import imread

/home/thallessilva/.virtualenvs/cv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Obs: If you only have one of the datasets (does not matter which one), just run all the notebook's cells and it will work just fine.

In [2]:
# define base paths for pascal the original VOC dataset training images
base_dataset_dir_voc = '<path-to-voc-2012>/PascalVoc2012/train/VOC2012'
images_folder_name_voc = "JPEGImages/"
annotations_folder_name_voc = "SegmentationClass_1D/"
images_dir_voc = os.path.join(base_dataset_dir_voc, images_folder_name_voc)
annotations_dir_voc = os.path.join(base_dataset_dir_voc, annotations_folder_name_voc)

In [3]:
# define base paths for pascal augmented VOC images
# download: http://home.bharathh.info/pubs/codes/SBD/download.html
base_dataset_dir_aug_voc = '<pascal/augmented/VOC/images/path>/benchmark_RELEASE/dataset'
images_folder_name_aug_voc = "img/"
annotations_folder_name_aug_voc = "cls/"
images_dir_aug_voc = os.path.join(base_dataset_dir_aug_voc, images_folder_name_aug_voc)
annotations_dir_aug_voc = os.path.join(base_dataset_dir_aug_voc, annotations_folder_name_aug_voc)

In [4]:
def get_files_list(base_dataset_dir, images_folder_name, annotations_folder_name, filename):
    images_dir = os.path.join(base_dataset_dir, images_folder_name)
    annotations_dir = os.path.join(base_dataset_dir, annotations_folder_name)

    file = open(filename, 'r')
    images_filename_list = [line for line in file]
    return images_filename_list

In [5]:
images_filename_list = get_files_list(base_dataset_dir_aug_voc, images_folder_name_aug_voc, annotations_folder_name_aug_voc, "custom_train.txt")
print("Total number of training images:", len(images_filename_list))

Total number of training images: 8252


In [6]:
# shuffle array and separate 10% to validation
np.random.shuffle(images_filename_list)
val_images_filename_list = images_filename_list[:int(0.10*len(images_filename_list))]
train_images_filename_list = images_filename_list[int(0.10*len(images_filename_list)):]

In [7]:
print("train set size:", len(train_images_filename_list))
print("val set size:", len(val_images_filename_list))

train set size: 7427
val set size: 825


In [8]:
TRAIN_DATASET_DIR="./tfrecords/"
if not os.path.exists(TRAIN_DATASET_DIR):
    os.mkdir(TRAIN_DATASET_DIR)
    
TRAIN_FILE = 'train.tfrecords'
VALIDATION_FILE = 'validation.tfrecords'
train_writer = tf.python_io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,TRAIN_FILE))
val_writer = tf.python_io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,VALIDATION_FILE))

In [9]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [10]:
def read_annotation_from_mat_file(annotations_dir, image_name):
    annotations_path = os.path.join(annotations_dir, (image_name.strip() + ".mat"))
    mat = spio.loadmat(annotations_path)
    img = mat['GTcls']['Segmentation'][0][0]
    return img

In [11]:
def create_tfrecord_dataset(filename_list, writer):

    # create training tfrecord
    read_imgs_counter = 0
    for i, image_name in enumerate(filename_list):

        try:
            image_np = imread(os.path.join(images_dir_aug_voc, image_name.strip() + ".jpg"))
        except FileNotFoundError:
            try:
                # read from Pascal VOC path
                image_np = imread(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))
            except FileNotFoundError:
                print("File:",image_name.strip(),"not found.")
                continue
        try:
            annotation_np = read_annotation_from_mat_file(annotations_dir_aug_voc, image_name)
        except FileNotFoundError:
            # read from Pascal VOC path
            try:
                annotation_np = imread(os.path.join(annotations_dir_voc, image_name.strip() + ".png"))
            except FileNotFoundError:
                print("File:",image_name.strip(),"not found.")
                continue
            
        read_imgs_counter += 1
        image_h = image_np.shape[0]
        image_w = image_np.shape[1]

        img_raw = image_np.tostring()
        annotation_raw = annotation_np.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(image_h),
                'width': _int64_feature(image_w),
                'image_raw': _bytes_feature(img_raw),
                'annotation_raw': _bytes_feature(annotation_raw)}))

        writer.write(example.SerializeToString())
    
    print("End of TfRecord. Total of image written:", read_imgs_counter)
    writer.close()

In [12]:
# create training dataset
create_tfrecord_dataset(train_images_filename_list, train_writer)

File: 2007_000720 not found.
File: 2007_006673 not found.
File: 2007_007447 not found.
File: 2007_007230 not found.
File: 2007_003668 not found.
File: 2007_005859 not found.
File: 2007_006400 not found.
File: 2008_006920 not found.
File: 2007_007481 not found.
File: 2009_000420 not found.
File: 2007_007930 not found.
File: 2007_001416 not found.
File: 2007_001609 not found.
File: 2007_001595 not found.
File: 2007_003876 not found.
File: 2007_007947 not found.
File: 2007_008945 not found.
File: 2007_008218 not found.
File: 2007_009832 not found.
File: 2007_003286 not found.
File: 2007_007355 not found.
File: 2007_002216 not found.
File: 2007_009597 not found.
File: 2007_001340 not found.
File: 2007_007773 not found.
File: 2007_003190 not found.
File: 2007_000243 not found.
File: 2007_006409 not found.
File: 2007_004065 not found.
File: 2007_004841 not found.
File: 2007_008043 not found.
File: 2007_006490 not found.
File: 2011_003151 not found.
File: 2007_004289 not found.
File: 2007_009

File: 2011_002717 not found.
File: 2007_006136 not found.
File: 2007_004500 not found.
File: 2007_001027 not found.
File: 2007_004627 not found.
File: 2007_009947 not found.
File: 2007_002896 not found.
File: 2007_008526 not found.
File: 2007_005227 not found.
File: 2007_001602 not found.
File: 2007_005314 not found.
File: 2007_008927 not found.
File: 2007_009139 not found.
File: 2007_004481 not found.
File: 2007_007621 not found.
File: 2007_002361 not found.
File: 2007_003565 not found.
File: 2007_001698 not found.
File: 2007_003910 not found.
File: 2007_006530 not found.
File: 2007_007098 not found.
File: 2007_005902 not found.
File: 2007_009295 not found.
End of TfRecord. Total of image written: 7120


In [13]:
# create validation dataset
create_tfrecord_dataset(val_images_filename_list, val_writer)

File: 2007_002234 not found.
File: 2007_000039 not found.
File: 2007_009605 not found.
File: 2007_000480 not found.
File: 2007_006303 not found.
File: 2007_001917 not found.
File: 2007_001225 not found.
File: 2007_008407 not found.
File: 2007_005144 not found.
File: 2007_001825 not found.
File: 2007_005043 not found.
File: 2007_003525 not found.
File: 2007_008714 not found.
File: 2007_009665 not found.
File: 2007_007890 not found.
File: 2007_002107 not found.
File: 2007_000250 not found.
File: 2007_001439 not found.
File: 2007_007891 not found.
File: 2007_000904 not found.
File: 2007_006581 not found.
File: 2009_002599 not found.
File: 2007_005262 not found.
End of TfRecord. Total of image written: 802
